<a href="https://colab.research.google.com/github/Aqeel151/Myproject/blob/master/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU langchain langchain_community langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash',verbose=True)

llm.invoke("Where is Country Pakiatan Situated and how pepole lived there")

AIMessage(content="Pakistan is situated in South Asia, bordering India to the east, Afghanistan to the west, Iran to the southwest, and China to the north.  It also has a coastline along the Arabian Sea to the south.\n\nDescribing how people live in Pakistan is complex, as it's a diverse country with significant regional variations in culture, lifestyle, and socioeconomic conditions.  There's no single answer.  However, some general observations can be made:\n\n* **Diverse Population:** Pakistan has a large and diverse population with different ethnic groups, languages, and religious beliefs.  The majority of the population is Muslim, but there are also significant minorities of Christians, Hindus, and others.  Major ethnic groups include Punjabis, Sindhis, Pashtuns, Balochs, and Muhajirs.  This diversity leads to a wide range of cultural practices and traditions.\n\n* **Urban vs. Rural Life:**  A significant portion of the population lives in urban areas, with large cities like Karach

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])


In [ ]:
tools_list=[multiply]
model_with_tools = llm.bind_tools(tools_list)

In [ ]:
result = llm_with_tools.invoke("Hey how are you!")
result

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-3d7ca9d2-61ae-4356-9a26-b2ede989e0cb-0', usage_metadata={'input_tokens': 68, 'output_tokens': 17, 'total_tokens': 85, 'input_token_details': {'cache_read': 0}})

In [ ]:
result = llm_with_tools.invoke("What is 2 multiplied by 3?")
result


AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 3.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-553b6eb6-b035-4d8a-987c-63d1e440171a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 3.0}, 'id': 'cd98157a-d82c-4c3b-844d-df44be523a72', 'type': 'tool_call'}], usage_metadata={'input_tokens': 72, 'output_tokens': 3, 'total_tokens': 75, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain.tools import Tool

# Example tool for addition
def add_numbers(a, b):
    return a * b

math_tool = Tool(
    name="AdditionTool",
    func=lambda x: f"The result is: {add_numbers(*map(int, x.split()))}",
    description="Adds two numbers. Input should be two integers separated by a space."
)


In [ ]:
from langchain.agents import initialize_agent, AgentType

# List of tools
tools = [math_tool]

# Initialize the agent
agent = initialize_agent(
    tools=tools,                        # Provide the tools
    llm=llm,                            # LLM for fallback
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent type
    verbose=True                        # Enable debugging output
)


In [ ]:
# Query the agent
response1 = agent.run("What is 10 * 10?")  # Uses the tool
print(response1)





> Entering new AgentExecutor chain...
Question: What is 10 * 10?
Thought: I need to multiply 10 by 10. I can do this by repeatedly adding 10 ten times.
Action: AdditionTool
Action Input: 10 10
Observation: The result is: 100
Thought:Question: What is 10 * 10?
Thought: I need to multiply 10 by 10. I can do this by repeatedly adding 10 ten times.  I'll use the AdditionTool nine more times.
Action: AdditionTool
Action Input: 10 10
Observation: The result is: 100
Thought:Question: What is 10 * 10?
Thought: I need to multiply 10 by 10. I can do this by repeatedly adding 10 ten times.  I will use the AdditionTool to add 10 to the running total nine more times.  I'll start with a running total of 100 from the first addition.
Action: AdditionTool
Action Input: 100 10
Observation: The result is: 1000
Thought:Question: What is 10 * 10?
Thought: I need to multiply 10 by 10. I can do this by repeatedly adding 10 ten times.
Action: AdditionTool
Action Input: 10 10
Observation: The result is: 100
